In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

In [ ]:
提取数据集,查看数据分布情况

In [ ]:
df=pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
label=df['label']
imgs=df.iloc[:,1:].values.reshape(-1,28,28)/255.
assert imgs.shape[0]==len(label)
print(imgs.shape)

In [ ]:
df['label'].value_counts().plot(kind='bar')

In [ ]:
alphabet=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
               'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
idx = np.random.randint(len(label))

plt.imshow(imgs[idx])
plt.title('The corresponding label is {}'.format(alphabet[label[idx]]))

TensorFlow data pipeline

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,Sequential,Model
label=np.array(label).astype('int8')
imgs =imgs.astype('float32')
imgs = imgs[...,tf.newaxis]

train_set=tf.data.Dataset.from_tensor_slices((imgs[:20000],label[:20000]))
train_set=train_set.shuffle(20000).batch(128).prefetch(1)
test_set=tf.data.Dataset.from_tensor_slices((imgs[20000:],label[20000:]))
test_set=test_set.batch(128)

In [ ]:
for img,l in train_set.take(1):
    plt.figure(figsize=(10,10))
    for idx in range(16):
        plt.subplot(4,4,idx+1)
        plt.imshow(np.squeeze(img[idx]))
        plt.title('The corresponding \nlabel is {}'.format(alphabet[l[idx]]))

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(25, activation = 'softmax'))
#if use CategoricalCrossentropy,we need to convert label to one-hot vector
model.compile(loss =tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics =['acc'])

In [ ]:
hist=model.fit(train_set,epochs=10,validation_data=test_set)

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = hist.history['acc']
val_acc = hist.history['val_acc']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

在测试集上查看模型预测效果

In [ ]:
df_test=pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')
label_test=df_test['label'].values
imgs_test=df_test.iloc[:,1:].values.reshape(-1,28,28)/255.

pred_labels=model.predict(imgs_test[...,tf.newaxis])
pred_labels=tf.argmax(pred_labels,axis=-1)

In [ ]:
errors=len(np.where(label_test!=pred_labels)[0])
print('model predict accuracy is :{:.2f}%'.format((1-errors/len(label_test))*100))

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(label_test, pred_labels)
print('Accuracy Score = ', accuracy)

i = np.random.randint(1,len(label_test))
plt.imshow(imgs_test[i]) 
print("Predicted Label: ", alphabet[int(pred_labels[i])])
print("True Label: ", alphabet[int(label_test[i])])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
#Print confusion matrix
cm = confusion_matrix(label_test, pred_labels)

fig, ax = plt.subplots(figsize=(12,12))
sns.set(font_scale=1.1)
sns.heatmap(cm, annot=True, linewidths=.5, ax=ax)


#PLot fractional incorrect misclassifications
incorr_fraction = 1 - np.diag(cm) / np.sum(cm, axis=1)
fig, ax = plt.subplots(figsize=(12,12))
plt.bar(np.arange(24), incorr_fraction)
plt.xlabel('True Label')
plt.ylabel('Fraction of incorrect predictions')
plt.xticks(np.arange(25), alphabet,rotation=0) 
plt.show()

# As we can see,the 'M','Q' is the most difficult image  for model to recognize